In [1]:
# mutliple outputs in cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# cell width
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

### Imports

In [2]:
import pandas as pd
import numpy as np

### Loading data

In [3]:
tokens = pd.read_parquet('tokens.parquet')
dependencies = pd.read_parquet('dependencies.parquet')
parses = pd.read_parquet('parses.parquet')

In [4]:
tokens
tokens.dtypes

,movie_id,sentence_id,token_id,word,lemma,COB,COE,POS,NER
0,10000053,1,1,Fur,Fur,0,3,NNP,O
1,10000053,1,2,trapper,trapper,4,11,NNP,O
2,10000053,1,3,Jean,Jean,12,16,NNP,PERSON
3,10000053,1,4,La,La,17,19,NNP,PERSON
4,10000053,1,5,B,B,20,21,NNP,PERSON
...,...,...,...,...,...,...,...,...,...
14905198,9999280,6,14,face,face,500,504,NN,O
14905199,9999280,6,15,reality,reality,505,512,NN,O
14905200,9999280,6,16,more,more,513,517,RBR,O
14905201,9999280,6,17,clearly,clearly,518,525,RB,O


movie_id        int32
sentence_id     int16
token_id        int16
word           string
lemma          string
COB             int16
COE             int16
POS            string
NER            string
dtype: object

In [5]:
dependencies
dependencies.dtypes

,movie_id,sentence_id,dependency_class,dependency_type,governor_id,governor_word,dependent_id,dependent_word
0,10000053,1,basic,nn,6,te,1,Fur
1,10000053,1,basic,nn,6,te,2,trapper
2,10000053,1,basic,nn,6,te,3,Jean
3,10000053,1,basic,nn,6,te,4,La
4,10000053,1,basic,nn,6,te,5,B
...,...,...,...,...,...,...,...,...
34199063,9999280,6,collapsed-ccprocessed,nn,15,reality,13,Marcelo
34199064,9999280,6,collapsed-ccprocessed,nn,15,reality,14,face
34199065,9999280,6,collapsed-ccprocessed,nsubj,17,clearly,15,reality
34199066,9999280,6,collapsed-ccprocessed,advmod,17,clearly,16,more


movie_id             int32
sentence_id          int16
dependency_class    string
dependency_type     string
governor_id          int16
governor_word       string
dependent_id         int16
dependent_word      string
dtype: object

In [6]:
parses
parses.dtypes

,movie_id,sentence_id,parse
0,10000053,1,(ROOT (S (NP (NNP Fur) (NNP trapper) (NNP Jean...
1,10000053,2,(ROOT (S (S (PP (IN At) (NP (DT the) (NN settl...
2,10000053,3,(ROOT (S (NP (DT The) (NN trader)) (VP (VBZ ex...
3,10000053,4,"(ROOT (S (ADVP (RB Later)) (, ,) (NP (DT the) ..."
4,10000053,5,(ROOT (S (S (NP (NNP Jean) (NNP La) (NNP B) (N...
...,...,...,...
665581,9999280,2,(ROOT (S (NP (PRP He)) (VP (VBZ 's) (ADJP (JJ ...
665582,9999280,3,(ROOT (S (NP (PRP He)) (VP (VP (VBZ lives) (PP...
665583,9999280,4,(ROOT (S (NP (PRP He)) (VP (VBZ devotes) (NP (...
665584,9999280,5,"(ROOT (S (S (NP (PRP She)) (, ,) (ADVP (RB how..."


movie_id        int32
sentence_id     int16
parse          string
dtype: object